# Forward Kinematics Python Examples

This notebook combines Python scripts from the forward kinematics section of the robotics document, demonstrating key concepts in spatial descriptions and transformations using NumPy and SciPy.

## 1. Rotation Matrices: Random Rotation Example

This section demonstrates creating a random rotation matrix using SciPy's Rotation class, and verifies its properties such as determinant and orthogonality.

In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# Create a random rotation
r = R.random()
R_matrix = r.as_matrix()
print("Rotation matrix:")
print(R_matrix)
print("Determinant:", np.linalg.det(R_matrix))
print("R^T R:", np.dot(R_matrix.T, R_matrix))

Rotation matrix:
[[-0.00959766  0.71130811 -0.70281481]
 [-0.95041157  0.21199891  0.2275397 ]
 [ 0.31084681  0.67014717  0.67400076]]
Determinant: 0.9999999999999999
R^T R: [[ 1.00000000e+00 -4.63790977e-17 -2.10738696e-17]
 [-4.63790977e-17  1.00000000e+00  9.11502205e-19]
 [-2.10738696e-17  9.11502205e-19  1.00000000e+00]]


## 2. Special Rotation Matrices

This section shows how to generate rotation matrices for rotations about the principal axes (X, Y, Z) using SciPy's Rotation class with rotation vectors.

In [2]:
# Rotation about x by 45 degrees
r_X = R.from_rotvec([np.pi/4, 0, 0])
print("R_X(45 degrees):")
print(r_X.as_matrix())

# Similarly for y and z
r_Y = R.from_rotvec([0, np.pi/4, 0])
r_Z = R.from_rotvec([0, 0, np.pi/4])

R_X(45 degrees):
[[ 1.          0.          0.        ]
 [ 0.          0.70710678 -0.70710678]
 [ 0.          0.70710678  0.70710678]]


## 3. Composition of Rotations

This section illustrates intrinsic and extrinsic composition of rotations. Intrinsic composition applies rotations relative to the body's current orientation, while extrinsic applies them relative to the fixed frame.

In [3]:
# Define two rotations
r1 = R.from_rotvec([np.pi/6, 0, 0])  # 30 degrees about x
r2 = R.from_rotvec([0, np.pi/4, 0])  # 45 degrees about y

# Intrinsic composition: [R] = [R1][R2]
r_intrinsic = r1 * r2
print("Intrinsic rotation matrix:")
print(r_intrinsic.as_matrix())

# Extrinsic composition: [R] = [R2][R1]
r_extrinsic = r2 * r1
print("Extrinsic rotation matrix:")
print(r_extrinsic.as_matrix())

Intrinsic rotation matrix:
[[ 7.07106781e-01 -2.77555756e-17  7.07106781e-01]
 [ 3.53553391e-01  8.66025404e-01 -3.53553391e-01]
 [-6.12372436e-01  5.00000000e-01  6.12372436e-01]]
Extrinsic rotation matrix:
[[ 7.07106781e-01  3.53553391e-01  6.12372436e-01]
 [-2.77555756e-17  8.66025404e-01 -5.00000000e-01]
 [-7.07106781e-01  3.53553391e-01  6.12372436e-01]]


## 4. Euler Angles Conversion

This section demonstrates converting Euler angles to a rotation matrix and back, showing the round-trip conversion for the XYZ sequence.

In [4]:
# Given Euler angles (alpha, beta, gamma) in radians
original_angles = np.array([0.1, 0.2, 0.3])  # example values

# Generate rotation matrix from Euler angles
r_original = R.from_euler('xyz', original_angles)
R_matrix = r_original.as_matrix()
print("Rotation matrix from original angles:")
print(R_matrix)

# Now, use inverse kinematics to extract Euler angles
extracted_angles = r_original.as_euler('xyz', degrees=False)
print("Extracted Euler angles (alpha, beta, gamma):")
print(extracted_angles)

# Check if they match (within numerical precision)
difference = np.abs(original_angles - extracted_angles)
print("Difference between original and extracted:")
print(difference)
print("Are they close?", np.allclose(original_angles, extracted_angles))

Rotation matrix from original angles:
[[ 0.93629336 -0.27509585  0.21835066]
 [ 0.28962948  0.95642509 -0.03695701]
 [-0.19866933  0.0978434   0.97517033]]
Extracted Euler angles (alpha, beta, gamma):
[0.1 0.2 0.3]
Difference between original and extracted:
[0.00000000e+00 1.66533454e-16 5.55111512e-17]
Are they close? True


## 5. Homogeneous Transformations

This section shows how to apply a homogeneous transformation matrix to transform a point from one coordinate frame to another.

In [5]:
# Homogeneous transformation matrix from frame B to frame A
A_T_B = np.array([
    [0.866, -0.500, 0.000, 10.0],
    [0.500,  0.866, 0.000,  5.0],
    [0.000,  0.000, 1.000,  0.0],
    [0.000,  0.000, 0.000,  1.0]
])

# Point P in frame B (homogeneous coordinates)
B_P = np.array([3.0, 7.0, 0.0, 1.0])

# Transform point into frame A
A_P = A_T_B @ B_P

# Extract just the 3D coordinates
print("\nCoordinates of P in frame A:")
print(A_P[:3])


Coordinates of P in frame A:
[ 9.098 12.562  0.   ]


## 6. Inverse and Compound Transformations

This section demonstrates composing two homogeneous transformations and computing the inverse of the compound transformation, then applying them to a point.

In [6]:
# Define two homogeneous transformations

# Transformation T1: rotation by 30 degrees around z-axis, translation [10, 5, 0]
r1 = R.from_euler('z', np.pi/6)
T1 = np.eye(4)
T1[:3, :3] = r1.as_matrix()
T1[:3, 3] = [10, 5, 0]

# Transformation T2: rotation by 45 degrees around x-axis, translation [2, 3, 1]
r2 = R.from_euler('x', np.pi/4)
T2 = np.eye(4)
T2[:3, :3] = r2.as_matrix()
T2[:3, 3] = [2, 3, 1]

# Compound transformation: T_compound = T1 @ T2
T_compound = T1 @ T2
print("Compound transformation matrix:")
print(T_compound)

# Inverse of the compound transformation
T_inv = np.linalg.inv(T_compound)
print("Inverse transformation matrix:")
print(T_inv)

# Example: Apply compound transformation to a point
P = np.array([1.0, 2.0, 3.0, 1.0])  # Point in homogeneous coordinates
P_transformed = T_compound @ P
print("Point after compound transformation:", P_transformed[:3])

# Apply inverse to get back
P_back = T_inv @ P_transformed
print("Point after applying inverse:", P_back[:3])

Compound transformation matrix:
[[ 0.8660254  -0.35355339  0.35355339 10.23205081]
 [ 0.5         0.61237244 -0.61237244  8.59807621]
 [ 0.          0.70710678  0.70710678  1.        ]
 [ 0.          0.          0.          1.        ]]
Inverse transformation matrix:
[[ 8.66025404e-01  5.00000000e-01  4.49653636e-17 -1.31602540e+01]
 [-3.53553391e-01  6.12372436e-01  7.07106781e-01 -2.35475540e+00]
 [ 3.53553391e-01 -6.12372436e-01  7.07106781e-01  9.40541835e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Point after compound transformation: [11.4516296   8.48570378  4.53553391]
Point after applying inverse: [1. 2. 3.]


## 7. Forward Kinematics of UR5e Robot Arm

This section demonstrates forward kinematics for the UR5e robot arm using the Modified Denavit-Hartenberg (DHM) parameters. The DHM convention uses parameters $a_i$, $\alpha_i$, $d_i$, and $\theta_i$, with positive link lengths preferred and offsets added to $\theta$ for negative lengths.

### DHM Parameters for UR5e

| Joint $i$ | $a_{i}$ [mm] | $\alpha_{i}$ [rad] | $d_{i}$ [mm] | $\theta_{i}$ [rad] |
|-----------|--------------|-------------------|--------------|-------------------|
| 1         | 0            | 0                 | 162.5        | 0                 |
| 2         | 0            | $\frac{\pi}{2}$   | 0            | $\pi$             |
| 3         | 425          | 0                 | 0            | 0                 |
| 4         | 392.25       | 0                 | 133.3        | 0                 |
| 5         | 0            | $-\frac{\pi}{2}$  | 99.7         | 0                 |
| 6         | 0            | $\frac{\pi}{2}$   | 99.6         | $\pi$             |

The kinematic diagram and DHM parameters are visualized below:

![UR5e Kinematic Diagram](sections/imgs/UR_DH.png)

![RoboDK Software](sections/imgs/RoboDK.png)

![UR5e in RoboDK](sections/imgs/UR5e_RoboDK.png)

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# DHM parameters for UR5e (in mm and rad)
a = [0, 0, 425, 392.25, 0, 0]
alpha = [0, np.pi/2, 0, 0, -np.pi/2, np.pi/2]
d = [162.5, 0, 0, 133.3, 99.7, 99.6]
theta_offsets = [0, np.pi, 0, 0, 0, np.pi]

# Given joint angles in degrees
joint_angles_deg = [0, -90, -90, 0, 90, 0]
joint_angles_rad = np.deg2rad(joint_angles_deg)

# Add offsets to joint angles
theta = joint_angles_rad + theta_offsets

# Function to compute DH transformation matrix
def dh_transform(a, alpha, d, theta):
    T = np.eye(4)
    # Trans_x(a)
    T[0, 3] = a
    # Rot_x(alpha)
    T[:3, :3] = R.from_rotvec([alpha, 0, 0]).as_matrix()
    # Trans_z(d)
    T_z = np.eye(4)
    T_z[2, 3] = d
    T = T @ T_z
    # Rot_z(theta)
    T_theta = np.eye(4)
    T_theta[:3, :3] = R.from_rotvec([0, 0, theta]).as_matrix()
    T = T @ T_theta
    return T

# Compute total transformation
T_total = np.eye(4)
for i in range(6):
    T_i = dh_transform(a[i], alpha[i], d[i], theta[i])
    T_total = T_total @ T_i

print("End-effector transformation matrix:")
print(T_total)

End-effector transformation matrix:
[[-2.22044605e-16  2.22044605e-16  1.00000000e+00  4.91850000e+02]
 [-1.00000000e+00 -1.22460635e-16 -2.22044605e-16 -1.33300000e+02]
 [ 1.22460635e-16 -1.00000000e+00  2.22044605e-16  6.87200000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
